In [24]:
import os

In [25]:
%pwd

'/Users/aneeshgrover/PROJECTS/Text-Summarizer-Project'

In [26]:
os.chdir("../")

In [27]:
%pwd

'/Users/aneeshgrover/PROJECTS'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [30]:
import sys
import os

# Add the src directory to the Python path
current_working_directory = os.getcwd()
src_directory = os.path.join(current_working_directory, '/Users/aneeshgrover/PROJECTS/Text-Summarizer-Project/src')
sys.path.insert(0, src_directory)

from textSummarizer.utils.common import read_yaml, create_directories


In [31]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [33]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [37]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /Users/aneeshgrover/PROJECTS


In [41]:
import logging
from pathlib import Path
import yaml
from typing import Any
import os

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Helper functions
def read_yaml(file_path: Path) -> Any:
    with file_path.open('r') as file:
        logger.info(f"Reading YAML file: {file_path}")
        return yaml.safe_load(file)

def create_directories(paths: list):
    for path in paths:
        os.makedirs(path, exist_ok=True)
        logger.info(f"Directory created or already exists: {path}")

# Check current working directory
current_working_directory = Path(os.getcwd()).resolve()
logger.info(f"Current working directory: {current_working_directory}")

# Define paths to YAML files
CONFIG_FILE_PATH = current_working_directory / 'config/config.yaml'
PARAMS_FILE_PATH = current_working_directory / 'params.yaml'

# Ensure the files exist before proceeding
if not CONFIG_FILE_PATH.exists():
    logger.error(f"Config file not found: {CONFIG_FILE_PATH}")
    raise FileNotFoundError(f"Config file not found: {CONFIG_FILE_PATH}")
if not PARAMS_FILE_PATH.exists():
    logger.error(f"Params file not found: {PARAMS_FILE_PATH}")
    raise FileNotFoundError(f"Params file not found: {PARAMS_FILE_PATH}")

logger.debug(f"Absolute path of CONFIG_FILE_PATH: {CONFIG_FILE_PATH}")
logger.debug(f"Absolute path of PARAMS_FILE_PATH: {PARAMS_FILE_PATH}")

# Define DataIngestionConfig (assuming it is a dataclass)
from dataclasses import dataclass

@dataclass
class DataIngestionConfig:
    root_dir: str
    source_URL: str
    local_data_file: str
    unzip_dir: str

# ConfigurationManager class definition
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        create_directories([config['root_dir']])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_URL=config['source_URL'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )
        return data_ingestion_config

# Example usage
try:
    config_manager = ConfigurationManager()
    logger.info("Configuration manager initialized successfully")
    logger.debug(config_manager.config)
    logger.debug(config_manager.params)

    artifacts_root = Path(config_manager.config['artifacts_root'])
    if artifacts_root.exists():
        logger.info(f"The directory '{artifacts_root}' was successfully created.")
    else:
        logger.warning(f"Failed to create the directory '{artifacts_root}'.")
except FileNotFoundError as e:
    logger.error(f"File not found: {e.filename}")
except Exception as e:
    logger.critical(f"An error occurred: {str(e)}")


[2024-06-10 22:15:30,720: INFO: 2403624338: Current working directory: /Users/aneeshgrover/PROJECTS]
[2024-06-10 22:15:30,723: ERROR: 2403624338: Config file not found: /Users/aneeshgrover/PROJECTS/config/config.yaml]


FileNotFoundError: Config file not found: /Users/aneeshgrover/PROJECTS/config/config.yaml

In [39]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: The file config/config.yaml does not exist.